In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from robust_active_learning.data.datamanager import Data_manager
from robust_active_learning.data.datahandler_for_array import create_dataloader
from robust_active_learning.data.deprecated.datahandler_for_array import create_dataloader as create_dataloader_old
from robust_active_learning.data.deprecated.datamanager import get_datamanager as get_datamanager_old
from robust_active_learning.data.deprecated.sampler import DDU_sampler as DDU_sampler_old

In [3]:
from robust_active_learning.experiment_ddu import experiment_ddu

basic_settings = {
"oracle_stepsize": 50,
"oracle_steps": 3,
"iD": "CIFAR10",
"OoD": ["FashionMNIST", "MNIST"],
"grayscale": False,
"subclass": {
    "do_subclass": False,
    "iD_classes": [],
    "OoD_classes": []
},
"labelled_size": 2000,
"pool_size": 15000,
"OOD_ratio": 0.15,
"epochs": 100,
"batch_size": 64,
"weight_decay": 1e-4,
"metric": "accuracy",
"lr": 0.1,
"nesterov": False,
"momentum": 0.9,
"lr_sheduler": True,
"num_classes": 10,
"validation_split": 0.1,
"validation_source": "test",
"criterion": "crossentropy",
"verbose": 1
}
exp_settings = {
    "exp_type": "ddu",
    "exp_name": "ddu-cifar-015",
    "plots": False,
    "model": "DDU",
    "oracle": "ddu-sampler",
    "spectral_normalization": True,
    "temp": 1.0
}


c:\Users\nk\Python-proj\SRP\SRPHildesheim2021\.venv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [4]:
from torch.utils.tensorboard import SummaryWriter
import time

def create_log_dirs(log_path):
    if os.path.exists(log_path) == False:
        os.makedirs(log_path)

    status_manager_path = os.path.join(log_path, "status_manager_dir")
    writer_path = os.path.join(log_path, "writer_dir")
    log_dir_path = os.path.join(log_path, "log_dir")

    if os.path.exists(status_manager_path) == False:
        os.mkdir(status_manager_path)
    if os.path.exists(writer_path) == False:
        os.mkdir(writer_path)
    if os.path.exists(log_dir_path) == False:
        os.mkdir(log_dir_path)

    print("Directories created")

log_path = os.path.join("./logs")

log_path = os.path.join(log_path, time.strftime("%m-%d-%H-%M", time.localtime()))
create_log_dirs(log_path)

writer = SummaryWriter(os.path.join(log_path, "writer_dir"))

Directories created


In [5]:
exp_ddu_old=experiment_ddu(basic_settings,exp_settings, log_path, writer)

In [6]:
exp_ddu_old.construct_datamanager()


INFO ------ List of datasets being loaded are ['CIFAR10', 'FashionMNIST', 'MNIST']
Files already downloaded and verified
Files already downloaded and verified
INFO ----- Dataset Loaded : CIFAR10
INFO ----- Dataset Loaded : MNIST
INFO ----- Dataset Loaded : FashionMNIST
INFO ----- Total iD samples for training  50000
INFO ----- Total iD samples for testing  10000
INFO ----- Total OoD samples for training 120000
initialised datamanager


AttributeError: 'Data_manager' object has no attribute 'status_manager'

In [8]:
exp_ddu_old.datamanager.create_merged_data()

Creating New Dataset
Status_manager intialised


In [10]:
exp_ddu_old.datamanager.status_manager.describe()

,inds,target,source,status
count,17000.000000,17000.000000,17000.000000,17000.000000
mean,28744.068529,8.625000,-0.500000,0.117647
std,16702.116379,2.781156,0.866051,0.322199
min,2.000000,0.000000,-1.000000,0.000000
25%,14420.250000,9.750000,-1.000000,0.000000
50%,28483.000000,10.000000,-1.000000,0.000000
75%,42823.000000,10.000000,-0.500000,0.000000
max,59997.000000,10.000000,1.000000,1.000000


In [8]:
exp_ddu_old=experiment_ddu(basic_settings,exp_settings, log_path, writer)
#exp_ddu_old.perform_experiment()

INFO ------ List of datasets being loaded are ['CIFAR10', 'FashionMNIST', 'MNIST']
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/100 [00:00<?, ?it/s]

INFO ----- Dataset Loaded : CIFAR10
INFO ----- Dataset Loaded : MNIST
INFO ----- Dataset Loaded : FashionMNIST
INFO ----- Total iD samples for training  50000
INFO ----- Total iD samples for testing  10000
INFO ----- Total OoD samples for training 120000
initialised datamanager
loaded statusmanager from file
Using Training data to create validation dataset, size : 200

Training with device : cuda:0
Number of Training Samples :  1800
Number of Validation Samples :  200
Number of Epochs :  100
Layer (type:depth-idx)                        Param #
├─BatchNorm2d: 1-1                            32
├─Conv2d: 1-2                                 432
├─Sequential: 1-3                             --
|    └─BasicBlock: 2-1                        --
|    |    └─Conv2d: 3-1                       2,304
|    |    └─BatchNorm2d: 3-2                  32
|    |    └─Conv2d: 3-3                       2,304
|    |    └─BatchNorm2d: 3-4                  32
|    |    └─Sequential: 3-5                   --
|

  1%|          | 1/100 [00:08<13:31,  8.20s/it]

Validation loss decreased (inf --> 2.634883).  Saving model ...
Val_loss: 2.6349 Val_acc : 18.50
Train_loss: 2.0720 Train_acc : 20.83

Epoch: 2


...

Validation loss decreased (inf --> 2.183149).  Saving model ...
Val_loss: 2.1831 Val_acc : 18.50
Train_loss: 2.0792 Train_acc : 22.32

Epoch: 2


  2%|▏         | 2/100 [00:18<14:46,  9.05s/it]

Validation loss decreased (2.183149 --> 1.990014).  Saving model ...
Val_loss: 1.9900 Val_acc : 35.00
Train_loss: 1.7936 Train_acc : 32.93

Epoch: 3


  2%|▏         | 2/100 [00:18<15:30,  9.49s/it]


KeyboardInterrupt: 

In [7]:
inds=[0,1,2,3,4]

In [9]:
exp_ddu_old.datamanager.status_manager[:5]

,target,source,status
0,1.0,1.0,1.0
1,3.0,1.0,1.0
2,4.0,1.0,1.0
3,2.0,1.0,1.0
4,5.0,1.0,1.0


In [15]:
exp_ddu_old.datamanager.status_manager.describe()

,target,source,status
count,19647.000000,19647.000000,19647.000000
mean,3.758996,0.730544,0.106887
std,3.265803,0.682883,0.317107
min,-1.000000,-1.000000,0.000000
25%,1.000000,1.000000,0.000000
50%,4.000000,1.000000,0.000000
75%,7.000000,1.000000,0.000000
max,9.000000,1.000000,2.000000


In [14]:
exp_ddu_old.datamanager.status_manager["status"].unique()

array([1., 2., 0.])

In [12]:
exp_ddu_old.datamanager.status_manager.iloc[:, -1].unique()

array([1., 2., 0.])

In [6]:
exp_ddu_old=experiment_ddu(basic_settings,exp_settings, log_path, writer)
exp_ddu_old.perform_old_experiment()

Files already downloaded and verified
Files already downloaded and verified
(50000, 3, 32, 32) (10000, 3, 32, 32) (140000, 3, 32, 32) (140000,)
Base-data shape:  (50000, 3, 32, 32)
OOD_data shape:  (140000, 3, 32, 32)
Creating New Dataset
Experiment_Setup saved
Status_manager intialised


  0%|          | 0/100 [00:00<?, ?it/s]

Using Testing data to create validation dataset, size : 1000

Training with device : cuda:0
Number of Training Samples :  2000
Number of Validation Samples :  1000
Number of Epochs :  100
INFO ------ Early Stopping Patience not specified using 10

Epoch: 1


  1%|          | 1/100 [00:07<11:51,  7.18s/it]

Validation loss decreased (inf --> 2.294787).  Saving model ...

Epoch: 2


...

Validation loss decreased (1.692452 --> 1.673795).  Saving model ...

Epoch: 10


 10%|█         | 10/100 [00:45<06:27,  4.30s/it]

Validation loss decreased (1.673795 --> 1.619933).  Saving model ...
Val_loss: 1.6199 Val_acc : 42.20
Train_loss: 1.5512 Train_acc : 43.71

Epoch: 11


 10%|█         | 10/100 [00:47<07:05,  4.73s/it]


KeyboardInterrupt: 